📘 Jupyter Notebook: Visualizing and Comparing Power System Graphs with GeoPandas

In [1]:
# 📦 Section 1: Imports & Setup
from pathlib import Path
import geopandas as gpd
import pandas as pd
from grid_reducer.utils import get_ckt_from_opendss_model
from grid_reducer.network import get_graph_from_circuit
from grid_reducer.plot import graph_to_geodataframe
from grid_reducer.reducer import OpenDSSModelReducer

In [2]:
# ⚡ Section 2: Load and Parse OpenDSS Circuit

# Specify the path to your DSS master file
master_dss_path = Path("../tests/data/smartds/Master.dss")

# Load the circuit object from the OpenDSS model
ckt = get_ckt_from_opendss_model(master_dss_path)
reducer = OpenDSSModelReducer(master_dss_file=master_dss_path)
reduced_ckt = reducer.reduce()

# Create a NetworkX graph from the circuit
#graph = get_graph_from_circuit(ckt)
graph = get_graph_from_circuit(reduced_ckt)

# Print basic info
print(f"Number of nodes: {len(graph.nodes)}")
print(f"Number of edges: {len(graph.edges)}")

Number of aggregated lines = 12
Number of removed lines = 24
Switch buses: set()
Transforming coordinates...
{'p12udt1266-p12uhs0_1247x': array([-0.47362126,  0.23428641]), 'p12udt2952': array([-0.40612729,  0.1544501 ]), 'p12udt2953': array([-0.40286351,  0.18664116]), 'p12udt2952-p12udt5990xx': array([-0.41769873,  0.12434794]), 'p12udt2655-p12udt2659x': array([-0.62087473, -0.39655792]), 'p12udt2659': array([-0.64385377, -0.41892187]), 'p12udt5620': array([-0.59215917, -0.51739021]), 'p12udt6806': array([-0.60667483, -0.54508443]), 'p12udt2644-p12udt5620x': array([-0.57831408, -0.48833597]), 'p12udt2661': array([-0.55716346, -0.42465526]), 'p12udt2644': array([-0.56609067, -0.45754336]), 'p12udt2654-p12udt2661x': array([-0.55472764, -0.38989995]), 'p12udt2651-p12udt2658x': array([-0.59493188, -0.32289606]), 'p12udt2658': array([-0.62417545, -0.33180642]), 'p12udt2950-p12udt2954xx': array([-0.94204558, -0.24861081]), 'p12udt2950-p12udt2954x': array([-0.97125992, -0.25672032]), 'p12ud

In [9]:
# 🗺️ Section 3: Convert Graph to GeoDataFrame for Mapping

# Convert NetworkX graph to GeoDataFrame
gdf = graph_to_geodataframe(graph)

# Preview the GeoDataFrame
gdf.head()

,edge_id,source,target,length_km,kv,component_type,name,geometry
0,0,0,121,3.49,7.199558,Line,line__56,"LINESTRING (-0.4939 0.24296, -0.48705 0.21228)"
1,1,1,2,5.86,7.199558,Line,line__67,"LINESTRING (-0.4038 0.1422, -0.4058 0.19496)"
2,2,1,3,3.10,7.199558,Line,line__128,"LINESTRING (-0.4038 0.1422, -0.40788 0.11458)"
3,3,3,119,5.18,7.199558,Line,line__15,"LINESTRING (-0.40788 0.11458, -0.45442 0.11173)"
4,4,4,123,2.57,7.199558,Line,line__75,"LINESTRING (-0.61119 -0.38761, -0.59003 -0.37822)"


In [10]:
# 🧭 Section 4: Explore the Graph with Interactive Map

# Visualize with geopandas.explore()
gdf.explore(
    style_kwds={
        "color": "red",  # Line color
        "weight": 5,  # Line width
        "opacity": 0.8,  # Line transparency (optional)
    },
    legend=True,
)

In [7]:
# 🔍 Section 5: Comparing Multiple Graphs Side-by-Side

# Load a second circuit (e.g., a modified version for comparison)
master_dss_path2 = Path("../tests/data/smartds/Master.dss")
ckt2 = get_ckt_from_opendss_model(master_dss_path2)
reducer = OpenDSSModelReducer(master_dss_file=master_dss_path)
reduced_ckt = reducer.reduce()
ckt2 = reduced_ckt
graph2 = get_graph_from_circuit(ckt2)
gdf2 = graph_to_geodataframe(graph2)

# Add 'category' column to both
gdf["source"] = "Base"
gdf2["source"] = "Variant"

# Merge for comparison
combined_gdf = pd.concat([gdf, gdf2], ignore_index=True)

# Visualize both overlaid, color-coded by source
combined_gdf.explore(column="source", legend=True, cmap="Set2")

Number of aggregated lines = 12
Number of removed lines = 24
Transforming coordinates...
{'p12udt1266-p12uhs0_1247x': array([-0.47362126,  0.23428641]), 'p12udt2952': array([-0.40612729,  0.1544501 ]), 'p12udt2953': array([-0.40286351,  0.18664116]), 'p12udt2952-p12udt5990xx': array([-0.41769873,  0.12434794]), 'p12udt2655-p12udt2659x': array([-0.62087473, -0.39655792]), 'p12udt2659': array([-0.64385377, -0.41892187]), 'p12udt5620': array([-0.59215917, -0.51739021]), 'p12udt6806': array([-0.60667483, -0.54508443]), 'p12udt2644-p12udt5620x': array([-0.57831408, -0.48833597]), 'p12udt2661': array([-0.55716346, -0.42465526]), 'p12udt2644': array([-0.56609067, -0.45754336]), 'p12udt2654-p12udt2661x': array([-0.55472764, -0.38989995]), 'p12udt2651-p12udt2658x': array([-0.59493188, -0.32289606]), 'p12udt2658': array([-0.62417545, -0.33180642]), 'p12udt2950-p12udt2954xx': array([-0.94204558, -0.24861081]), 'p12udt2950-p12udt2954x': array([-0.97125992, -0.25672032]), 'p12udt2951-p12udt2954x': 